# BigQuery Read/Write with GDS, Spark Connector and PySpark
In the examples that follows, we will be using the Spark Connector running under PySpark
[Neo4j spark connector under Python](https://neo4j.com/docs/spark/current/python/) and the Graph Data Science (GDS) client.

Please run this notebook from a valid Spark environment.  It was tested under [DataProc](https://cloud.google.com/dataproc).

## Setup Neo4j instance
Create a free account at https://sandbox.neo4j.com. Choose the “Blank Sandbox - Graph Data Science” option.
When your sandbox has been created, fill in the Bolt URL and password below.

## Environment Setup

In [ ]:
pip install --upgrade seaborn

In [ ]:
pip install --upgrade matplotlib

## Setup Neo4j Spark Connector imports

In [ ]:
from graphdatascience import GraphDataScience
from pyspark.sql import SparkSession
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
from pyspark import SparkFiles

Define Neo4j connection variables.  Yours will be different.

In [ ]:
neo4j_url = "bolt://***removed***:7687"
neo4j_user = "neo4j"
neo4j_password = "***removed***"
neo4j_database= "neo4j"
tmp_working_bucket = "neo4j-sandbox/dataproc-working"

Create Spark Session, seeded with Neo4j packages.  If you don't want to wait for the download each time, load the connector into the master node using SSH.

Note: we are adding the Neo4j data warehouse connector and BigQuery library in the library packages.

In [ ]:
spark = (SparkSession.builder
        .appName('Leverage Neo4j with Apache Spark')
        .master('local[*]')
        # Just to show dataframes as tables
        .config('spark.sql.repl.eagerEval.enabled', True)
        # On DataProc we must use spark 2.x
        .config('spark.jars.packages', "org.neo4j:neo4j-connector-apache-spark_2.12:4.1.3_for_spark_2.4,com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.26.0")
        # These global credentials don't cascade on DataProc
        .config('neo4j.url', neo4j_url)
        .config('neo4j.authentication.type', "basic")
        .config('neo4j.authentication.basic.username', neo4j_user)
        .config('neo4j.authentication.basic.password', neo4j_password)
        .getOrCreate())
# output spark version
spark

Let's create a handle to the Graph Data Science library so we can make GDS calls and scripts more elegantly.

In [ ]:
gds = GraphDataScience(neo4j_url, auth=(neo4j_user, neo4j_password))

Let's now check that GDS is running on the server by executing this Cypher query.
We only need to supply credentials once per notebook.

In [ ]:
gds.run_cypher("return gds.version() as gds_version")

Query to see what data is there already using Spark Connector

In [ ]:
spark.read.format("org.neo4j.spark.DataSource") \
    .option("url", neo4j_url) \
    .option('authentication.type', "basic") \
    .option('authentication.basic.username', neo4j_user) \
    .option('authentication.basic.password', neo4j_password) \
    .option("labels", "MSA").load().show()

If dataset is not empty, reset it.

In [ ]:
gds.run_cypher("CREATE OR REPLACE DATABASE `"+neo4j_database+"`")

Check that it's empty now.

In [ ]:
gds.run_cypher("MATCH (n:MSA) RETURN count(n)")

# Load MSA data from Text

First, let's try downloading a file from URL the Spark way

In [ ]:
url = "https://raw.githubusercontent.com/smithna/datasets/main/CensusDemographicsByMetroArea.csv"
spark.sparkContext.addFile(url)
urlDf = spark.read.csv("file://"+SparkFiles.get("CensusDemographicsByMetroArea.csv"), header=True, inferSchema= True)
urlDf.count()

Check to see what kind of job Spark did inferring schema

In [ ]:
urlDf.printSchema()

We see that percentOver25WithBachelors is a double, which is not supported in Neo4j 4.x
We can cast to float using selectExpr.

In [ ]:
floatCastSelectExpr=["NAME as name","population",\
"cast(PERCENTOVER25WITHBACHELORS as float) percentOver25WithBachelors",\
"cast(MEDIANHOMEPRICE as int) medianHomePrice",\
"cast(MEDIANHOUSEHOLDINCOME as int) medianHouseholdIncome"]

castUrlDf = urlDf.selectExpr(floatCastSelectExpr)
castUrlDf.printSchema()

Now delete it, we are loading now from BigQuery

In [ ]:
gds.run_cypher("CREATE OR REPLACE DATABASE `"+neo4j_database+"`")

In [ ]:
gds.run_cypher("MATCH (n:MSA) RETURN count(n)")

# Load MSA data from BigQuery

This section leverages best practices described here: https://neo4j.com/docs/spark/current/dwh/

Create unique constraint on MSA.  The column "name" is the msa name.

In [ ]:
gds.run_cypher("""
CREATE CONSTRAINT msa_name IF NOT EXISTS ON (m:MSA) ASSERT m.name IS NODE KEY
""")

In [ ]:
Create json template for reading and writing input from BigQuery. The data is here:

    select name,
       population,
       medianHomePrice
       percentOver25WithBachelors,
       medianHouseholdIncome
    from census.census_demos_by_msa

## BigQuery: Parallelized table scan
The preferred approach for querying BigQuery is a simple table scan.
This method supports maxParallelization if you want to throttle, otherwise it will maximize speed.

https://github.com/GoogleCloudDataproc/spark-bigquery-connector/blob/master/README.md

In [ ]:
# If we're doing a table scan, the options table syntax can be used and it provides multi-threading
msa_table="""
    neo4jbusinessdev.census.msa_demos
"""
tableBqDf = spark.read.format("bigquery").option("table",msa_table).load()

Let's count the records

In [ ]:
tableBqDf.count()

Now show the results

In [ ]:
tableBqDf.show()

### BigQuery: Post query filtering
We loaded all MSAs in massive parallel, now we can filter down the recordset using Spark SQL.

In [ ]:
metro_select="""
SELECT * FROM msa_df WHERE name LIKE "%Metro%"
"""
tableBqDf.createOrReplaceTempView("msa_df")
bigqueryPostQueryDf = spark.sql(metro_select)

In [ ]:
bigqueryPostQueryDf.count()

In [ ]:
bigqueryPostQueryDf.show()

## BigQuery: SQL push-down, temporary materialization
To push down a full SQL statement to BigQuery, we must materialize of results in a BigQuery dataset.
The temporary materialization is used as a cache by BigQuery.  It will be deleted after 1 day by default.

In [ ]:
tempLocation = "dataproc_testing"
spark.conf.set("viewsEnabled","true")
spark.conf.set("materializationDataset","census")
# if this value is too low, big query will expect that it exists in cache.  Default is 1440.
spark.conf.set("materializationExpirationTimeInMinutes",5)

msa_query_sql="""
    select * from `neo4jbusinessdev.census.msa_demos` where name LIKE "%Metro%"
    """
bigqueryDf = spark.read.format("bigquery").option("query",msa_query_sql).load()

Count results

In [ ]:
bigqueryDf.count()

Show results

In [ ]:
bigqueryDf.show()

In [ ]:
bigqueryDf.printSchema()

Let's cast columns before committing to Neo4j

In [ ]:
floatCastSelectExpr=["NAME as name","population",\
"cast(PERCENTOVER25WITHBACHELORS as float) percentOver25WithBachelors",\
"cast(MEDIANHOMEPRICE as int) medianHomePrice",\
"cast(MEDIANHOUSEHOLDINCOME as int) medianHouseholdIncome"]

castBigqueryDf = bigqueryDf.selectExpr(floatCastSelectExpr)

In [ ]:
castBigqueryDf.printSchema()

The Neo4j Spark Connector makes it easy to load data from a DataFrame into Neo4j.

In [ ]:
# This write is Spark native
(castBigqueryDf.write
  .format("org.neo4j.spark.DataSource")
  .mode("Overwrite")
  .option("labels", ":MSA")
  .option("node.keys","name")
  .option("url", neo4j_url) 
  .option("authentication.type", "basic") 
  .option("authentication.basic.username", neo4j_user) 
  .option("authentication.basic.password", neo4j_password) 
  .option("partitions", "1") 
  .save())

Check for data now

In [ ]:
gds.run_cypher("MATCH (n:MSA) RETURN count(n)")

Read from Neo4j

In [ ]:
neo4jDf=spark.read.format("org.neo4j.spark.DataSource") \
  .option("labels", "MSA") \
  .option("url", neo4j_url) \
  .option("authentication.type", "basic")  \
  .option("authentication.basic.username", neo4j_user)  \
  .option("authentication.basic.password", neo4j_password)  \
  .load()

In [ ]:
neo4jDf.printSchema()

Cast double to long.  Renaming could be done in the same step.

In [ ]:
cast_sql="""
    select name, 
    cast(POPULATION as long) population,
    cast(PERCENTOVER25WITHBACHELORS as float) percentOver25WithBachelors,
    cast(MEDIANHOMEPRICE as int) medianHomePrice,
    cast(MEDIANHOUSEHOLDINCOME as int) medianHouseholdIncome
    from msa_df
"""
neo4jDf.createOrReplaceTempView("msa_df")
neo4jDfCasted = spark.sql(cast_sql)
neo4jDfCasted.printSchema()

Convert Spark dataframe to pandas to display histogram

In [ ]:
pandas_msa_df=neo4jDfCasted.toPandas()
print(pandas_msa_df)

Now show histograms of data

In [ ]:
fig, axes = plt.subplots(4, 2)
fig.set_size_inches(15,30)
for i in range(1,5):
    sns.histplot(pandas_msa_df.iloc[:,i], ax=axes[i-1,0])
    sns.histplot(pandas_msa_df.iloc[:,i], log_scale=True, ax=axes[i-1,1])

That log transformation looks like it will help. Run the Cypher to store the transformed values in the graph.

In [ ]:
log_new_view_sql = """
SELECT name, 
population, CAST(log(population)  as float) as logPopulation,
medianHouseholdIncome,  CAST(log(medianHouseholdIncome) as float) as logMedianHouseholdIncome,
medianHomePrice, CAST(log(medianHomePrice) as float) as logMedianHomePrice,
CAST(percentOver25WithBachelors as float) as percentOver25WithBachelors, CAST(log(percentOver25WithBachelors) as float) as logPercentOver25WithBachelors
FROM msa_df
"""
neo4jDfCasted.createOrReplaceTempView("msa_df")
neo4jDfLogUpdated = spark.sql(log_new_view_sql)
neo4jDfLogUpdated.printSchema()

Commit to the database

In [ ]:
# This write is Spark native
(neo4jDfLogUpdated.write
  .format("org.neo4j.spark.DataSource")
  .mode("Overwrite")
  .option("labels", ":MSA")
  .option("node.keys","name")
  .option("url", neo4j_url) 
  .option("authentication.type", "basic") 
  .option("authentication.basic.username", neo4j_user) 
  .option("authentication.basic.password", neo4j_password) 
  .option("partitions", "1") 
  .save())

Look in the database

In [ ]:
neo4jDf=spark.read.format("org.neo4j.spark.DataSource") \
  .option("labels", "MSA") \
  .option("url", neo4j_url) \
  .option("authentication.type", "basic")  \
  .option("authentication.basic.username", neo4j_user)  \
  .option("authentication.basic.password", neo4j_password)  \
  .load().printSchema()

Check that logs were committed to the database

## Create in-memory graph projection
Passing `"*"` as the third argument to `gds.graph.project` tells GDS to include any relationships that exist in the database in the in-memory graph. Because no relationships have been created in the graph yet, there will be no relationships in the in-memory graph projection when it is created.

In [ ]:
g_msa, result = gds.graph.project(
    'msa-graph', 
    'MSA', 
    '*', 
    nodeProperties = [
        "logPopulation", 
        "logMedianHouseholdIncome", 
        "logMedianHomePrice", 
        "logPercentOver25WithBachelors"])


Notice that when we look at the results of gds.graph.project, we see that the relationshipCount is 0.

In [ ]:
result

## Apply MinMax scalar to property values

In [ ]:
gds.alpha.scaleProperties.mutate(g_msa, 
                                 nodeProperties = [
                                     "logPopulation", 
                                     "logMedianHouseholdIncome", 
                                     "logMedianHomePrice", 
                                     "logPercentOver25WithBachelors"], 
                                 scaler = "MinMax",
                                 mutateProperty = "scaledProperties")

This next line streams node properties to the procedure caller.

In [ ]:
sp = gds.graph.streamNodeProperty(g_msa, "scaledProperties")

In [ ]:
import pandas as pd

Cleanup resources

In [ ]:
pd.DataFrame(list(sp['propertyValue'])).iloc[:,0].hist()

In [ ]:
pd.DataFrame(list(sp['propertyValue'])).iloc[:,1].hist()

In [ ]:
pd.DataFrame(list(sp['propertyValue'])).iloc[:,2].hist()

In [ ]:
pd.DataFrame(list(sp['propertyValue'])).iloc[:,3].hist()

## Run KNN to create relationships to nearest neighbors
First run in stats mode and look at the similarity distribution.

In [ ]:
knn_stats = gds.knn.stats(g_msa,
                          nodeProperties={"scaledProperties":"EUCLIDEAN"},
                          topK=15
                         )

In [ ]:
knn_stats['similarityDistribution']

Now run KNN in mutate mode to update the in-memory graph projection. We'll exclude the bottom 1% of similarity relationships.

In [ ]:
gds.knn.mutate(g_msa,
               nodeProperties={"scaledProperties":"EUCLIDEAN"},
               topK=15,
               mutateRelationshipType="IS_SIMILAR",
               mutateProperty="similarity",
               similarityCutoff=knn_stats['similarityDistribution']['p1']
              )

Also write the relationships from the in-memory graph projection back to the on-disk graph.

In [ ]:
gds.graph.writeRelationship(
    g_msa,
    "IS_SIMILAR",
    relationship_property="similarity"
)

Add a `rank` property to the `IS_SIMILAR` relationships for use with Bloom filtering.

In [ ]:
gds.run_cypher("""
MATCH (m:MSA)-[s:IS_SIMILAR]->()
WITH m, s ORDER BY s.similarity DESC
WITH m, collect(s) as similarities, range(0, 19) AS ranks
UNWIND ranks AS rank
WITH rank, similarities[rank] AS rel
SET rel.rank = rank + 1
""")

## Run Louvain Community Detection

See how many communities Louvain is going to recommend

In [ ]:
gds.louvain.stats(g_msa,
                  relationshipTypes=["IS_SIMILAR"],
                 relationshipWeightProperty="similarity"
                 )

Now commit louvain communities to database

In [ ]:
gds.louvain.write(g_msa,
                  relationshipTypes=["IS_SIMILAR"],
                  relationshipWeightProperty ="similarity",
                  writeProperty="communityId")

## Gather statistics about the communities that were discovered

Get average values for each community and 3 example MSAs for each community.

In [ ]:
communityDf = gds.run_cypher("""
MATCH (m:MSA)
WITH m 
ORDER BY apoc.coll.sum([(m)-[s:IS_SIMILAR]->(m2) 
WHERE m.communityId = m2.communityId | s.similarity]) desc
RETURN m.communityId as communityId,
count(m) as msaCount, 
avg(m.population) as avgPopulation,
avg(m.medianHomePrice) as avgHomePrice,
avg(m.medianHouseholdIncome) as avgIncome,
avg(m.percentOver25WithBachelors) as avgPctBachelors,
collect(m.name)[..3] as exampleMSAs
""")

In [ ]:
communityDf.sort_values('communityId')

In [ ]:
fig, axes = plt.subplots(5, 1)
fig.set_size_inches(6,20)
for i in range(1,6):
    sns.barplot(data=communityDf, x="communityId", y=communityDf.columns[i], ax=axes[i-1])

Mean can give us a quick overview of properties, but can be skewed by outliers. Compare emperical cumulative distribution function (ECDF) at various proportions to get a more complete picture of distributions.

In [ ]:
detailDf = gds.run_cypher("""
MATCH (m:MSA)
RETURN "community " + m.communityId as communityId,
m.population as population,
m.medianHomePrice as medianHomePrice,
m.medianHouseholdIncome as medianIncome,
m.percentOver25WithBachelors as pctBachelors
order by m.communityId
""")

In [ ]:
fig, axes = plt.subplots(4, 1)
fig.set_size_inches(6,20)
for i in range(1,5):
    sns.ecdfplot(data=detailDf, hue="communityId", x=detailDf.columns[i], log_scale=True, ax=axes[i-1])

Compare two-dimensions on scatter plots

In [ ]:
splot = sns.scatterplot(data=detailDf, x="medianIncome", y="population", hue="communityId")
splot.set(yscale="log")
splot.set(xscale="log")

In [ ]:
splot = sns.scatterplot(data=detailDf, x="pctBachelors", y="medianHomePrice", hue="communityId")
splot.set(yscale="log")
splot.set(xscale="log")

## Write enriched features back to BigQuery

In this section, we are going to write features back to BigQuery.  Let's begin by previewing a query that we will materialize to BigQuery.  Note that you can use SKIP and LIMIT in cypher when using the Spark connector since it is batching the query internally.

In [ ]:
relationship_cypher="""
MATCH (s)-[r:IS_SIMILAR]-(t) 
RETURN s.name AS msa_name
,t.name AS related_name
,r.similarity AS similarity
,s.communityId AS louvain_community_id
"""
relsPandaDf=gds.run_cypher(relationship_cypher)
relsPandaDf

Convert pandas dataframe to Spark dataframe

In [ ]:
#Create PySpark DataFrame from Pandas
relsDf=spark.createDataFrame(relsPandaDf) 

Now write to bigquery.  Note that writes require a temporary storage location for Avro files in process.

In [ ]:
# Create Spark dataframe from 

# tmp_working_bucket
spark.conf.set("temporaryGcsBucket", "neo4j_sandbox")

(relsDf.write.format("bigquery")
  .mode("overwrite") 
  .option("table","neo4jbusinessdev.census.msa_demo_similarity")
  .save())

Now look for data in BigQuery

In [ ]:
spark.read.format("bigquery").option("table","census.msa_demo_similarity").load().show

## Optional: assign human-friendly names to the clusters discovered.
The Louvain community detection algorithm is not deterministic. You should have roughly the same clusters from previous runs, but some edge cases might be assigned to different communities. The community numbers might be shuffled between across different runs.  
**This step requires adjustment by hand: choose from community IDs above.**

In [ ]:
gds.run_cypher("""
MATCH (m:MSA) 
  SET m.communityName = CASE m.communityId 
  WHEN 54 THEN "Large mid-cost metros"
  WHEN 75 THEN "College towns"
  WHEN 81 THEN "Large high-cost metros"
  WHEN 234 THEN "Mid-size metros"
  WHEN 264 THEN "Small metros"
  WHEN 330 THEN "Mid-price metros"
  WHEN 385 THEN "Low-income metros"
  END
return m.communityName, m.communityId, count(*)
""")

Create an index on the communityName property to make it searchable in Bloom.

In [ ]:
gds.run_cypher("""
CREATE INDEX msa_community_name IF NOT EXISTS
FOR (m:MSA)
ON (m.communityName)
""")

Now open Bloom and do some additional analysis!

## Cleanup

Drop graph

In [ ]:
graph_project_drop = """
    CALL gds.graph.drop(
    'msa-graph')
"""
gds.run_cypher(graph_project_drop)

Drop BigQuery table.  There is no native commands to delete with spark so let's use the Python API.

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()
table_id = 'neo4jbusinessdev.census.msa_demo_similarity'
# Will raisegoogle.api_core.exceptions.NotFound unless not_found_ok is True.
client.delete_table(table_id, not_found_ok=True)  # Make an API request.
print("Deleted table '{}'.".format(table_id))


We can verify deletion here...  Error is expected!

In [ ]:
spark.read.format("bigquery").option("table","census.msa_demo_similarity").load().show()